### 1- Load The Data ??

In [1]:
from langchain_community.document_loaders import UnstructuredURLLoader

In [2]:
urls = [
    'https://www.victoriaonmove.com.au/local-removalists.html',
    'https://victoriaonmove.com.au/index.html',
    'https://victoriaonmove.com.au/contact.html'
]

loader = UnstructuredURLLoader(urls=urls)

data = loader.load()

In [3]:
data

[Document(metadata={'source': 'https://www.victoriaonmove.com.au/local-removalists.html'}, page_content='Loading...\n\nLOCAL REMOVALS\n\nYour trusted partner in seamless moving and packing solutions!\n\nGoogle logo G\n\nGoogle Rating\n\nGold star icon\n\nGold star icon\n\nGold star icon\n\nGold star icon\n\nGold star icon\n\n5 stars, 111 reviews\n\n\n\nRequst A call for You:\n\nLocal removal services via "Victoria on move"\n\nVictoria on Move is your trusted local moving company in Melbourne, specializing in seamless relocation services. As experienced furniture movers and relocation experts, we provide top-notch packing and moving services tailored to your needs. Whether you\'re moving across town or relocating interstate, our professional movers ensure a stress-free experience. Count on Victoria on Move for reliable removal services, making us the preferred choice among local movers in Melbourne. Discover why we\'re recognized for our commitment to quality and customer satisfaction.\

### 2- Split The Data Using Langchain Text Splitter ??

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Split Data
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

docs = text_splitter.split_documents(data)

print("Total Number of Documents -> ", len(docs))

Total Number of Documents ->  11


In [5]:
docs[0]

Document(metadata={'source': 'https://www.victoriaonmove.com.au/local-removalists.html'}, page_content='Loading...\n\nLOCAL REMOVALS\n\nYour trusted partner in seamless moving and packing solutions!\n\nGoogle logo G\n\nGoogle Rating\n\nGold star icon\n\nGold star icon\n\nGold star icon\n\nGold star icon\n\nGold star icon\n\n5 stars, 111 reviews\n\n\n\nRequst A call for You:\n\nLocal removal services via "Victoria on move"\n\nVictoria on Move is your trusted local moving company in Melbourne, specializing in seamless relocation services. As experienced furniture movers and relocation experts, we provide top-notch packing and moving services tailored to your needs. Whether you\'re moving across town or relocating interstate, our professional movers ensure a stress-free experience. Count on Victoria on Move for reliable removal services, making us the preferred choice among local movers in Melbourne. Discover why we\'re recognized for our commitment to quality and customer satisfaction.\n

### 3- Prepare The VectorDB and VectorStore ??

In [ ]:
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_groq import ChatGroq

In [7]:
from dotenv import load_dotenv
import os

load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [8]:
vectorstore = Chroma.from_documents(
    documents=docs,
    embedding=OllamaEmbeddings(model="deepseek-r1:1.5b")
)

C:\Users\hp\AppData\Local\Temp\ipykernel_5240\163251867.py:3: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding=OllamaEmbeddings(model="deepseek-r1:1.5b")


In [10]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

In [11]:
retrieved_docs = retriever.invoke("What kind of services they provided?")

In [12]:
len(retrieved_docs)

3

In [13]:
retrieved_docs

[Document(id='3c37af12-5bb7-40d5-8e19-4500d9830d0e', metadata={'source': 'https://www.victoriaonmove.com.au/local-removalists.html'}, page_content='Icon\n\nApartment Moving\n\nEfficient and careful relocation services tailored for apartments of all sizes, ensuring smooth transitions to your new home.\n\nIcon\n\nVilla Moving\n\nComprehensive moving solutions for large residences and villas, handling valuable possessions with utmost care and precision.\n\nIcon\n\nHousehold Moving\n\nFull-service moving options for households, including packing, loading, transportation, and unpacking services to simplify your move.\n\nIcon\n\nOffice Moving\n\nSpecialized expertise in office relocations, minimizing downtime and ensuring your business operations continue seamlessly.\n\nIcon\n\nFurniture Moving\n\nExperienced in handling furniture of all sizes and types, ensuring safe transport and setup in your new location.\n\nIcon\n\nPacking and Unpacking Services\n\nOptional packing and unpacking service

In [14]:
print(retrieved_docs[0].page_content)

Icon

Apartment Moving

Efficient and careful relocation services tailored for apartments of all sizes, ensuring smooth transitions to your new home.

Icon

Villa Moving

Comprehensive moving solutions for large residences and villas, handling valuable possessions with utmost care and precision.

Icon

Household Moving

Full-service moving options for households, including packing, loading, transportation, and unpacking services to simplify your move.

Icon

Office Moving

Specialized expertise in office relocations, minimizing downtime and ensuring your business operations continue seamlessly.

Icon

Furniture Moving

Experienced in handling furniture of all sizes and types, ensuring safe transport and setup in your new location.

Icon

Packing and Unpacking Services

Optional packing and unpacking services available to save you time and effort, using high-quality packing materials.

Icon

Customized Moving Plans


### 4- Prepare LLM ??

In [15]:
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0.4,
    max_tokens=500
)

### 5- Let's Create The Retrieval Chain and Start the Process Retrieving docs and Generating the Answer ??

In [16]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [18]:
system_prompt = (
    "You are an assistant for question-answering tasks."
    "Use the following pieces of retrieved context to answer"
    "the question. If you don't know the answer, say that you"
    "don't know. Use three sentences maximum and keep the"
    "answer concise."
    "\n\n"
    "{context}"
)

In [19]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
)

In [20]:
question_answer_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt
)

In [21]:
rag_chain = create_retrieval_chain(
    retriever,
    question_answer_chain
)

In [22]:
response = rag_chain.invoke({
    "input": "What kind of services they provide?"
})

In [23]:
print(response["answer"])

Victoria On Move provides various moving services, including apartment, villa, household, office, and furniture moving, as well as packing and unpacking services. They also offer customized moving plans and logistics solutions. Additionally, they have specialized services like furniture removals and provide insurance coverage for goods in transit.
